testes

In [1]:
from operator import contains
import os
import gzip
import tarfile
import json
import pandas as pd
import requests
from pathlib import Path
import io


In [6]:
# Define constants
raw_dir = Path("../data/raw")
extract_dir = Path("../data/extracted")
URLS = {
    'orders.json.gz': "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz",
    'consumers.csv.gz': "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/consumer.csv.gz",
    'restaurants.csv.gz': "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/restaurant.csv.gz",
    'ab_test.tar.gz': "https://data-architect-test-source.s3-sa-east-1.amazonaws.com/ab_test_ref.tar.gz"
}


#==============================
# Download files
#==============================
def download_file(url: str, filename:str, dir:Path=raw_dir):
    filepath = dir / filename

    response = requests.get(url, stream=True)
    response.raise_for_status()    

    with open(filepath, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    return filepath

download_file('https://data-architect-test-source.s3-sa-east-1.amazonaws.com/order.json.gz','orders.json.gz')

WindowsPath('../data/raw/orders.json.gz')

In [21]:
# Caminho do arquivo original
input_path = '../data/raw/orders.json.gz'

# Caminho para salvar o resultado processado
output_path = '../data/extracted/orders.parquet'

# Criar diretório de destino se não existir
os.makedirs(os.path.dirname(output_path), exist_ok=True)

# Ler o arquivo .gz e processar JSON linha por linha
records = []
with gzip.open(input_path, 'rt', encoding='utf-8') as f:
    for line in f:
        try:
            # Parsear cada linha como um objeto JSON
            record = json.loads(line.strip())
            records.append(record)
        except json.JSONDecodeError as e:
            print(f"Erro ao processar linha: {e}")

# Converter para DataFrame
df = pd.DataFrame(records)

# Salvar como parquet (mais eficiente para dados tabulares)
df.to_parquet(output_path, index=False)

print(f"Dados processados e salvos em {output_path}")

Salvou chunk 0 com 100000 registros
Salvou chunk 1 com 100000 registros
Salvou chunk 2 com 100000 registros
Salvou chunk 3 com 100000 registros
Salvou chunk 4 com 100000 registros
Salvou chunk 5 com 100000 registros
Salvou chunk 6 com 100000 registros
Salvou chunk 7 com 100000 registros
Salvou chunk 8 com 100000 registros
Salvou chunk 9 com 100000 registros
Salvou chunk 10 com 100000 registros
Salvou chunk 11 com 100000 registros
Salvou chunk 12 com 100000 registros
Salvou chunk 13 com 100000 registros
Salvou chunk 14 com 100000 registros
Salvou chunk 15 com 100000 registros
Salvou chunk 16 com 100000 registros
Salvou chunk 17 com 100000 registros
Salvou chunk 18 com 100000 registros
Salvou chunk 19 com 100000 registros
Salvou chunk 20 com 100000 registros
Salvou chunk 21 com 100000 registros
Salvou chunk 22 com 100000 registros
Salvou chunk 23 com 100000 registros
Salvou chunk 24 com 100000 registros
Salvou chunk 25 com 100000 registros
Salvou chunk 26 com 100000 registros
Salvou chun

In [7]:
df = pd.read_parquet("../data/extracted/restaurants.parquet")
df.head()

,id,created_at,enabled,price_range,average_ticket,takeout_time,delivery_time,minimum_order_value,merchant_zip_code,merchant_city,merchant_state,merchant_country
0,d19ff6fca6288939bff073ad0a119d25c0365c407e9e5d...,2017-01-23T12:52:30.910Z,False,3,60.0,0,50.0,30.0,14025,RIBEIRAO PRETO,SP,BR
1,631df0985fdbbaf27b9b031a8f381924e3483833385748...,2017-01-20T13:14:48.286Z,True,3,60.0,0,0.0,30.0,50180,SAO PAULO,SP,BR
2,135c5c4ae4c1ec1fdc23e8c649f313e39be8db913d8bc5...,2017-01-23T12:46:33.457Z,True,5,100.0,0,45.0,10.0,23090,RIO DE JANEIRO,RJ,BR
3,d26f84c470451f752bef036c55517b6d9950d41806f10e...,2017-01-20T13:15:04.806Z,True,3,80.0,0,0.0,18.9,40255,SALVADOR,BA,BR
4,97b9884600ea7192314580d9115f8882b8634f5aa201ff...,2017-01-20T13:14:27.701Z,True,3,60.0,0,0.0,25.0,64600,BARUERI,SP,BR


In [8]:

df = pd.read_parquet("../data/extracted/ab_test.parquet")
df.head()


,customer_id,is_target
0,755e1fa18f25caec5edffb188b13fd844b2af8cf5adedc...,target
1,b821aa8372b8e5b82cdc283742757df8c45eecdd72adf4...,control
2,d425d6ee4c9d4e211b71da8fc60bf6c5336b2ea9af9cc0...,control
3,6a7089eea0a5dc294fbccd4fa24d0d84a90c1cc12e829c...,target
4,dad6b7e222bab31c0332b0ccd9fa5dbd147008facd268f...,control


In [1]:
from pyspark.sql import SparkSession

In [5]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("GzipJsonProcessor") \
    .config("spark.sql.files.maxPartitionBytes", "128m") \
    .config("spark.default.parallelism", "8") \
    .config("spark.sql.shuffle.partitions", "8") \
    .config("spark.executor.memory", "6g") \
    .config("spark.driver.memory", "6g") \
    .config("spark.memory.offHeap.enabled", "true") \
    .config("spark.memory.offHeap.size", "2g") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .getOrCreate()

In [7]:
# Use the pre-created SparkSession to read and process the gzip file
df_spark = spark.read \
    .option("compression", "gzip") \
    .option("multiLine", "false") \
    .option("mode", "PERMISSIVE") \
    .json("../data/raw/orders.json.gz")
# Write directly to parquet with Spark

df_spark.write.mode("overwrite").parquet("../data/extracted/orders." + 'parquet')

# If you need a pandas DataFrame as the return value
df = df_spark.toPandas
df.head()

Py4JJavaError: An error occurred while calling o72.parquet.
: java.lang.RuntimeException: java.io.FileNotFoundException: java.io.FileNotFoundException: HADOOP_HOME and hadoop.home.dir are unset. -see https://wiki.apache.org/hadoop/WindowsProblems
	at org.apache.hadoop.util.Shell.getWinUtilsPath(Shell.java:735)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:270)
	at org.apache.hadoop.util.Shell.getSetPermissionCommand(Shell.java:286)
	at org.apache.hadoop.fs.RawLocalFileSystem.setPermission(RawLocalFileSystem.java:978)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkOneDirWithMode(RawLocalFileSystem.java:660)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:700)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.java:672)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirsWithOptionalPermission(RawLocalFileSystem.java:699)
	at org.apache.hadoop.fs.RawLocalFileSystem.mkdirs(RawLocalFileSystem.ja